# Data Science in Practice - Scraping - 1001

## Scraping Ascents of Routes. 

Author : Eoghan Cunningham

In [1]:
import pandas as pd
import urllib.request
import numpy as np
import matplotlib.pyplot as plt
import os
import os.path
import re
from bs4 import BeautifulSoup
from datetime import datetime
import scrapy
from loguru import logger
from scrapy.crawler import CrawlerProcess

Below are the necessary links to access route and climber details respectively. As you can see I've them up so they can be formatted with the route or climber ID and I tested it quickly with my own ID.

In [2]:
ukc_climbers_dir = '../data/raw/'
route_link = 'https://www.ukclimbing.com/logbook/c.php?i={}'
graphs_link = 'https://www.ukclimbing.com/logbook/showgraph.php?id={}'
crag_link = 'https://www.ukclimbing.com/logbook/crag.php?id={}'
my_climber_id = '220094'
british_grades = ['S','HS','VS','HVS','E1','E2','E3','E4','E5','E6','E7','E8']
graphs_link.format(my_climber_id)

'https://www.ukclimbing.com/logbook/showgraph.php?id=220094'

To begin scraping, we define a dict of the crags (locations) at which we wish to consider routes. 
I have collected the IDs used to identify each of the crags we plan to use.

In [3]:
crags = {
    'fair_head' : '17029',
    'dalkey_quarry' : '2337',
    'dinas_cromlech' : '4',
    'gogarth_south_stack' : '612',
    'gogarth_north_stack' : '598',
    'vivian_quarry' : '639',
    'rainbow_slab' : '637',
    'st_govans_head' : '7',
    'stennis_head' : '429',
    'rhoscolyn': '617',
    'curbar_edge' : '21',
    'millstone_edge' : '19',
    'cloggy': '457',
    'tremadog' : '221'
}

## Scraping Routes - get_routes()

The following function is written to take the crag ID and produce a dataframe of all of the relevant ascents at this crag. 

### Filtering : 

**Why?**

The function filters many of the routes from each location. It is important that the filtering takes place at this stage in the process to avoid unneccesary scraping later in the data retrieval proccess.

**How?**

Routes that are not considered to have a *significant* number of ascents will not be used. We do not believe we can make solid inferences from routes with less than 100 ascents. Such routes are removed. 

Routes with lower grades are far less likely to contain unsuccessful attempts. Such unsuccessful attempts will prove the basis for answering our research questions about grading. Routes with easier grades also contain far more ascents due to their accessibility. As a result scraping such routes will slow down the data retrieval proccess and the resulting data will prove less useful. 

In [4]:
def get_routes(crag_id) : 
    
    # format the crag link to produce the url, request it and use BS to parse the resulting html
    link = crag_link.format(crag_id)
    source = urllib.request.urlopen(link).read()
    soup = BeautifulSoup(source,'html.parser')

    # the name of the crag is stored here 
    location = soup.select('#breadcrumb-container > div > div.nav-tabs-header > h1')[0].text

    # each table row represents a route at the location. 
    table_rows = soup.find_all("tr")
    
    rows = []
    
    # iterate over these rows, skipping the first as it is a header. 
    for tr in table_rows[1:]:
        td = tr.find_all('td')
        info = [tr.text for tr in td] 
        # the route-id and number of ascents are not found in the text and need to be retieved manually.
        info.extend([tr.get("data-id"),tr.nextSibling]) 
        # if the grade for the route is not british, the row may contain 7 values. We pad the rest out with blank values.
        rows.append(info + [''] * (7 - len(info)))


    df = pd.DataFrame(rows[2:], columns=["index", "name", "info", "grade", "route_id", "num_ascents", "extra"])
    
    # the number of ascents is sometimes stored in a messy html tag, we use a regex to clean it
    df['num_ascents'] = df['num_ascents'].apply(lambda x : np.nan if x is None else re.sub(r'<.*?>', r'',str(x)))
    # and then cast it to an int
    df['num_ascents'] = df['num_ascents'].apply(lambda x : int(x) if str(x).isnumeric() else 0)

    # filtering based on the number of ascents. The most popular route on the site has 5,800 ascents. 
    df = df[df['num_ascents']>=100]
    df = df[df['num_ascents']<=6000]

    # we need to drop any grades that are not british. 
    # these are either sport routes or boulder problems and cannot be compared. 
    df['grade'] = df['grade'].apply(lambda string: re.split(r' +', str(string))[1] \
                                    if any(grade in string for grade in british_grades) else np.nan)

    # we drop unneccessary columns and any routes with no grades.
    df = df.drop(['index','info','extra'], axis=1).dropna(subset = ['grade'])

    df['location'] = location
    
    return df

The following calls the above **get_routes** function for each crag (location) in our crags dictionary, incrementally adding the results to our routes dataframe.   

In [5]:
routes = pd.DataFrame(columns=['name','grade','route_id','num_ascents','location'])

logger.debug('Getting routes from {} crags'.format(len(crags)))

for crag_name, crag_id in crags.items():
    logger.debug('Getting routes from {}'.format(crag_name))
    routes = routes.append(get_routes(crag_id))

routes

2019-05-10 21:40:11.288 | DEBUG    | __main__:<module>:3 - Getting routes from 14 crags
2019-05-10 21:40:11.291 | DEBUG    | __main__:<module>:6 - Getting routes from fair_head
2019-05-10 21:40:13.883 | DEBUG    | __main__:<module>:6 - Getting routes from dalkey_quarry
2019-05-10 21:40:20.129 | DEBUG    | __main__:<module>:6 - Getting routes from dinas_cromlech
2019-05-10 21:40:20.766 | DEBUG    | __main__:<module>:6 - Getting routes from gogarth_south_stack
2019-05-10 21:40:21.683 | DEBUG    | __main__:<module>:6 - Getting routes from gogarth_north_stack
2019-05-10 21:40:22.591 | DEBUG    | __main__:<module>:6 - Getting routes from vivian_quarry
2019-05-10 21:40:23.604 | DEBUG    | __main__:<module>:6 - Getting routes from rainbow_slab
2019-05-10 21:40:24.681 | DEBUG    | __main__:<module>:6 - Getting routes from st_govans_head
2019-05-10 21:40:30.367 | DEBUG    | __main__:<module>:6 - Getting routes from stennis_head
2019-05-10 21:40:34.892 | DEBUG    | __main__:<module>:6 - Getting 

,name,grade,route_id,num_ascents,location
110,Hurricane,E2,36923,119,Fair Head
120,Toby Jug,E1,51520,129,Fair Head
124,Jolly Roger,E3,36926,105,Fair Head
172,Burn Up,E1,36944,169,Fair Head
224,An Bealach Rúnda,E1,38806,146,Fair Head
303,Blind Pew,E2,53347,117,Fair Head
305,Mizen Star,E2,53349,101,Fair Head
311,Equinox,E2,53355,144,Fair Head
328,Hell's Kitchen,HVS,51528,246,Fair Head
334,Aoife,E2,36938,125,Fair Head


We can consider the size of the routes dataframe. 

In [6]:
len(routes)

410

In [7]:
routes['num_ascents'].sum()

174223

Of the above ascents many will not public and a further subset will not be 'Lead' attempts.

Next we have a function to produce a *pandas dataframe* listing all the ascents of a route given its ID.

In [8]:
def get_route_ascents(name, grade, location, route_id) : 

    def correct_dates(date) :
        
        # ascents from this year have no year given ie. they take the form DD/MM
        # we use datetime to give the current year (so this should still work in future)
        if date[-1].isalpha() :
            return date+", "+str(datetime.now().year)
        
        return date
        

    link = route_link.format(route_id)
    source = urllib.request.urlopen(link).read()
    soup = BeautifulSoup(source,'html.parser')
    
    # beautiful soup makes it easy to find the table and store all of it rows
    table = soup.find('div', attrs={'id':'public_logbooks'}).find('table')
    table_rows = table.find_all('tr', attrs={'class':''})
        
    ascents = []
    
    for tr in table_rows:
        td = tr.find_all('td')
        #the climber id is contained in the link to their logbook so we need to store that
        link = tr.find('a').get('href')
        info = [tr.text for tr in td]
        #then we can use a regex to grab it
        climber_id = re.sub(r'.*=', r'', link)
        info.append(climber_id)
        ascents.append(info)
        
        
    # we can convert the ascents data we've just collected into a dataframe
    df = pd.DataFrame(ascents, columns=["name", "date", "style", "comment", "climber_id"])
    
    
    # empty values are represented with a dash, we want to replace these with np.nan
    df['style'] = np.where(df['style']=='-', np.nan, df['style'])
    
    # and drop these null values
    df = df.dropna(subset=['style'])
    
    # finally we can fix the dates
    df['date'] = df["date"].apply(correct_dates)
    
    df['name'] = name
    df['grade'] = grade
    df['location'] = location
    
    return df
    

The following function is used to produce a full list of ascents from our routes dataframe by applying the **get_route_ascents** function to each route and collecting the results in a new dataframe. 

In [9]:
def get_ascents(routes) : 
    
    ascents = pd.DataFrame(columns=['name', 'date', 'style', 'comment', 'climber_id', 'grade', 'location'])
    for _, row in routes.iterrows():
        ascents = ascents.append(get_route_ascents(row['name'], row['grade'], row['location'], row['route_id']))

    return ascents

In [10]:
ascents = get_ascents(routes)
ascents

,name,date,style,comment,climber_id,grade,location
0,Hurricane,"20 Apr, 2019",2nd O/S,with Flavio,197782,E2,Fair Head
1,Hurricane,"20 Apr, 2019",Lead O/S,In one massive victory pitch. Puts all hand ja...,185136,E2,Fair Head
2,Hurricane,"15 Oct, 2018",Lead O/S,Another brilliant FH E2. Lead both pitches and...,57132,E2,Fair Head
3,Hurricane,"27 Jul, 2018",Lead,with Ali,63985,E2,Fair Head
4,Hurricane,"19 Jul, 2018",Lead O/S,Led both pitch,233566,E2,Fair Head
5,Hurricane,"7 Jul, 2018",AltLd O/S,with Robert Duran,133563,E2,Fair Head
6,Hurricane,"25 Jun, 2018",AltLd O/S,Great splitter climbing.with Mike Hutton,12933,E2,Fair Head
7,Hurricane,"4 Jun, 2018",AltLd O/S,Amazing! Led p2with Aggie,158515,E2,Fair Head
8,Hurricane,"3 Jun, 2018",AltLd,P2 easy finish. Really good fun.with Paul Collins,186094,E2,Fair Head
9,Hurricane,"3 Jun, 2018",AltLd O/S,Pitch 1with bren,151893,E2,Fair Head


In [11]:
len(ascents)

121247

We can see we've already lost many ascents as our function did not scrape 'hidden' ascents. Such ascents contain minimal data (just the date) and we don't believe anything can be learned from them. 

Further, we wish only to consider 'Lead' attempts. In particular we wish to exclude '2nd' or 'Top rope' ascents. In these cases the 2nd climber was essentially brought up the route by the leader. The 2nd climber would have been able to avoid much of the difficulty and danger of the route and thus, the ascent is not valid. 

In [12]:
leads = (ascents['style'].str.contains('Lead ')) | (ascents['style'].str.contains('AltLd '))
lead_ascents = ascents[leads]
len(lead_ascents)

60243

Finally we can store the dataset of all of the ascents.

In [13]:
#lead_ascents.to_csv("../data/ascents_data.csv")